# BibTeX Merge and DOI De-duplication

This notebook combines multiple BibTeX (`.bib`) files into a single tabular dataset.

I use DOI as the primary key because it is the most stable identifier across databases and export tools. Titles, authors, and even years vary in formatting, but a DOI usually does not. To avoid ambiguity, I filter to records with a usable DOI, normalize those DOIs, and de-duplicate strictly on the normalized DOI value.

The output is a single, traceable dataset that preserves source provenance while eliminating duplicate records.

In [1]:
from __future__ import annotations

import re
from pathlib import Path
from typing import Dict, Any, List, Tuple

import pandas as pd

## Parsing approach

I use `bibtexparser` to read BibTeX files because it handles common BibTeX structures and edge cases more reliably than a custom parser. Each BibTeX entry is converted into a row, and the originating `.bib` filename is preserved for later provenance and duplicate analysis.

In [2]:
import bibtexparser

NOTEBOOK_DIR = Path.cwd()
BIB_GLOB = "*.bib"

bib_paths = sorted(NOTEBOOK_DIR.glob(BIB_GLOB))

if not bib_paths:
    raise FileNotFoundError(
        f"No .bib files found in {NOTEBOOK_DIR}. "
        "Place the five .bib files in the same folder as this notebook."
    )

bib_paths

[WindowsPath('c:/Users/JanMc/Dropbox/Education/_GitHub_coursework/janmcconnellCityU-coursework/DS687_CAPSTONE/research/CIKM.bib'),
 WindowsPath('c:/Users/JanMc/Dropbox/Education/_GitHub_coursework/janmcconnellCityU-coursework/DS687_CAPSTONE/research/KDD.bib'),
 WindowsPath('c:/Users/JanMc/Dropbox/Education/_GitHub_coursework/janmcconnellCityU-coursework/DS687_CAPSTONE/research/RecSys.bib'),
 WindowsPath('c:/Users/JanMc/Dropbox/Education/_GitHub_coursework/janmcconnellCityU-coursework/DS687_CAPSTONE/research/TheWebConf.bib'),
 WindowsPath('c:/Users/JanMc/Dropbox/Education/_GitHub_coursework/janmcconnellCityU-coursework/DS687_CAPSTONE/research/WSDM.bib')]

In [3]:
DOI_CLEAN_RE = re.compile(r"\s+")
DOI_PREFIX_RE = re.compile(r"^(?:doi:\s*)", flags=re.IGNORECASE)
DOI_URL_PREFIX_RE = re.compile(r"^(?:https?://)?(?:dx\.)?doi\.org/", flags=re.IGNORECASE)

def normalize_doi(raw: Any) -> str | None:
    """
    Normalize DOI strings so the same DOI from different sources compares equal.

    Rules:
    - Convert to string, strip whitespace
    - Remove common prefixes like 'doi:' and 'https://doi.org/'
    - Remove internal whitespace
    - Lowercase
    - Keep only content that looks like a DOI when possible

    Returns:
    - normalized DOI string, or None if empty/unusable
    """
    if raw is None:
        return None

    s = str(raw).strip()
    if not s:
        return None

    s = DOI_PREFIX_RE.sub("", s).strip()
    s = DOI_URL_PREFIX_RE.sub("", s).strip()

    # Remove whitespace inside DOI just in case
    s = DOI_CLEAN_RE.sub("", s)

    s = s.lower()

    # Basic sanity check, DOI usually contains a slash
    if "/" not in s:
        return None

    # Remove trailing punctuation that often sneaks in
    s = s.rstrip(" .;,)}]>\n\t")

    return s or None

## Load records and preserve source filename

Each BibTeX entry becomes one row in a dataframe.  
I attach the source `.bib` filename to every record so duplicates can be traced back to their original exports.

In [4]:
def read_bib_file(path: Path) -> List[Dict[str, Any]]:
    with path.open("r", encoding="utf-8", errors="replace") as f:
        bib_db = bibtexparser.load(f)
    entries = bib_db.entries or []

    # Attach source filename
    for e in entries:
        e["_source_file"] = path.name
    return entries

all_entries: List[Dict[str, Any]] = []
for p in bib_paths:
    all_entries.extend(read_bib_file(p))

df_raw = pd.DataFrame(all_entries)

df_raw.shape, df_raw.columns.tolist()[:20]

((1371, 19),
 ['series',
  'location',
  'keywords',
  'numpages',
  'pages',
  'booktitle',
  'abstract',
  'doi',
  'url',
  'address',
  'publisher',
  'isbn',
  'year',
  'title',
  'author',
  'ENTRYTYPE',
  'ID',
  '_source_file',
  'articleno'])

## Filter to DOI-only records

To keep the dataset consistent and defensible, I restrict the analysis to records with a usable DOI after normalization. Records without a DOI are excluded from downstream analysis and de-duplication.

In [5]:
# Common BibTeX DOI field is 'doi', but sometimes it may appear as 'DOI' or other variants.
# I will look for 'doi' first, then fall back to a case-insensitive match.

def get_doi_series(df: pd.DataFrame) -> pd.Series:
    if "doi" in df.columns:
        return df["doi"]
    # fallback: find any column named like doi (case-insensitive)
    for c in df.columns:
        if str(c).strip().lower() == "doi":
            return df[c]
    # if none found, create empty series
    return pd.Series([None] * len(df), index=df.index)

df = df_raw.copy()
df["_doi_raw"] = get_doi_series(df)
df["_doi_norm"] = df["_doi_raw"].apply(normalize_doi)

df_doi = df[df["_doi_norm"].notna()].copy()

df_raw.shape, df_doi.shape

((1371, 19), (1342, 21))

## Exploratory counts by source file

I generate record counts per source file, including:
- total records per file
- records with a usable DOI per file

These counts confirm that each file was read correctly and show how DOI coverage varies across exports.

In [6]:
counts_total_per_file = (
    df_raw.groupby("_source_file")
    .size()
    .sort_values(ascending=False)
    .rename("records_total")
    .to_frame()
)

counts_doi_per_file = (
    df_doi.groupby("_source_file")
    .size()
    .sort_values(ascending=False)
    .rename("records_with_doi")
    .to_frame()
)

counts_by_file = counts_total_per_file.join(counts_doi_per_file, how="left").fillna(0).astype({"records_with_doi": "int64"})
counts_by_file

,records_total,records_with_doi
_source_file,,
RecSys.bib,379,371
CIKM.bib,309,309
TheWebConf.bib,304,298
KDD.bib,245,232
WSDM.bib,134,132


## Missing field counts within the DOI-only subset

Within the DOI-only subset, I count missing values for key bibliographic fields such as title, abstract, and year.

A field is treated as missing if it is absent, null, or an empty string after trimming. This highlights metadata quality differences across sources.

In [7]:
KEY_FIELDS = [
    "title",
    "abstract",
    "year",
    "author",
    "journal",
    "booktitle",
    "publisher",
    "volume",
    "number",
    "pages",
    "keywords",
]

def is_missing_value(x: Any) -> bool:
    if x is None:
        return True
    s = str(x).strip()
    return s == "" or s.lower() == "none" or s.lower() == "nan"

missing_counts = {}
for field in KEY_FIELDS:
    if field not in df_doi.columns:
        missing_counts[field] = len(df_doi)
    else:
        missing_counts[field] = df_doi[field].apply(is_missing_value).sum()

missing_report = pd.DataFrame(
    {"missing_count": missing_counts}
).sort_values("missing_count", ascending=False)

missing_report

,missing_count
number,1342
volume,1342
journal,1342
keywords,41
pages,11
abstract,6
title,0
year,0
publisher,0
author,0


## Duplicate DOI report across files

This step identifies DOIs that appear in more than one source file.

For each duplicated DOI, I report:
- total occurrences
- the list of source files in which it appears

This makes overlap across searches and filters explicit.

In [8]:
dup_cross_file = (
    df_doi.groupby("_doi_norm")
    .agg(
        total_occurrences=("_doi_norm", "size"),
        distinct_source_files=("_source_file", lambda s: sorted(set(s))),
        source_file_count=("_source_file", lambda s: len(set(s))),
    )
    .reset_index()
)

dup_cross_file_multi = dup_cross_file[dup_cross_file["source_file_count"] >= 2].copy()
dup_cross_file_multi = dup_cross_file_multi.sort_values(["source_file_count", "total_occurrences"], ascending=False)

dup_cross_file_multi

,_doi_norm,total_occurrences,distinct_source_files,source_file_count


## De-duplicate using DOI as the unique key

I retain exactly one record per normalized DOI.

When the same DOI appears multiple times, I keep the record with the most populated key fields, such as title, abstract, and year. If there is a tie, I keep the first occurrence. This favors richer metadata without manual intervention.

In [9]:
FIELDS_FOR_QUALITY = [f for f in KEY_FIELDS if f in df_doi.columns]

def quality_score(row: pd.Series) -> int:
    score = 0
    for f in FIELDS_FOR_QUALITY:
        score += 0 if is_missing_value(row.get(f, None)) else 1
    return score

df_doi["_quality_score"] = df_doi.apply(quality_score, axis=1)

# Sort so best quality comes first per DOI
df_doi_sorted = df_doi.sort_values(
    by=["_doi_norm", "_quality_score"],
    ascending=[True, False]
)

df_dedup = df_doi_sorted.drop_duplicates(subset=["_doi_norm"], keep="first").copy()

df_doi.shape, df_dedup.shape

((1342, 22), (1342, 22))

## Preserve provenance after de-duplication

After de-duplication, I retain provenance information by storing the full list of source files in which each DOI appeared, along with a count of distinct source files.

In [10]:
doi_to_sources = (
    df_doi.groupby("_doi_norm")["_source_file"]
    .apply(lambda s: sorted(set(s)))
    .to_dict()
)

df_dedup["_source_files_all"] = df_dedup["_doi_norm"].map(doi_to_sources)
df_dedup["_source_file_count"] = df_dedup["_source_files_all"].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Quick check
df_dedup[["_doi_norm", "_source_file", "_source_file_count", "_source_files_all"]].head(10)

,_doi_norm,_source_file,_source_file_count,_source_files_all
345,10.1109/asonam49781.2020.9381314,KDD.bib,1,[KDD.bib]
450,10.1109/asonam55673.2022.10068628,KDD.bib,1,[KDD.bib]
391,10.1109/asonam55673.2022.10068664,KDD.bib,1,[KDD.bib]
291,10.1145/1031171.1031201,CIKM.bib,1,[CIKM.bib]
1229,10.1145/1060745.1060754,TheWebConf.bib,1,[TheWebConf.bib]
538,10.1145/1081870.1081945,KDD.bib,1,[KDD.bib]
530,10.1145/1150402.1150508,KDD.bib,1,[KDD.bib]
1198,10.1145/1242572.1242806,TheWebConf.bib,1,[TheWebConf.bib]
550,10.1145/1281192.1281206,KDD.bib,1,[KDD.bib]
932,10.1145/1297231.1297235,RecSys.bib,1,[RecSys.bib]


## Optional exploratory checks

These additional counts support literature triage:
- publication year distribution
- top journals or venues
- records missing both title and abstract

All results are reported as counts rather than percentages.

In [11]:
# Year distribution (count)
if "year" in df_dedup.columns:
    year_counts = (
        df_dedup["year"]
        .apply(lambda x: str(x).strip() if not is_missing_value(x) else None)
        .dropna()
        .value_counts()
        .sort_index()
    )
    display(year_counts.to_frame("count"))

# Venue counts (journal, then booktitle)
venue_field = None
for candidate in ["journal", "booktitle"]:
    if candidate in df_dedup.columns:
        venue_field = candidate
        break

if venue_field:
    venue_counts = (
        df_dedup[venue_field]
        .apply(lambda x: str(x).strip() if not is_missing_value(x) else None)
        .dropna()
        .value_counts()
        .head(25)
    )
    display(venue_counts.to_frame(f"top_{venue_field}_count"))

# Low utility: missing both title and abstract
missing_title = df_dedup["title"].apply(is_missing_value) if "title" in df_dedup.columns else pd.Series([True]*len(df_dedup), index=df_dedup.index)
missing_abstract = df_dedup["abstract"].apply(is_missing_value) if "abstract" in df_dedup.columns else pd.Series([True]*len(df_dedup), index=df_dedup.index)

low_utility_count = (missing_title & missing_abstract).sum()
low_utility_count

,count
year,
1999,1
2001,2
2002,1
2003,2
2004,1
2005,2
2006,1
2007,8
2008,12


,top_booktitle_count
booktitle,
Proceedings of the 32nd ACM International Conference on Information and Knowledge Management,33
Proceedings of the 18th ACM Conference on Recommender Systems,32
Proceedings of the 17th ACM Conference on Recommender Systems,28
Proceedings of the 34th ACM International Conference on Information and Knowledge Management,27
Proceedings of the 31st ACM International Conference on Information &amp; Knowledge Management,26
Proceedings of the 14th ACM Conference on Recommender Systems,26
Proceedings of the 8th ACM Conference on Recommender Systems,24
Proceedings of the 30th ACM International Conference on Information &amp; Knowledge Management,24
The World Wide Web Conference,22


np.int64(0)

## Export combined dataset

I export a single CSV containing the combined, de-duplicated DOI dataset. The file includes normalized DOI, raw DOI, retained source file, all contributing source files, and all available BibTeX fields.

In [12]:
OUTPUT_CSV = NOTEBOOK_DIR / "bib_combined_dedup_doi.csv"

# Choose a stable column order: provenance and DOI columns first, then the rest
front_cols = [
    "_doi_norm",
    "_doi_raw",
    "_source_file",
    "_source_file_count",
    "_source_files_all",
    "_quality_score",
]

remaining_cols = [c for c in df_dedup.columns if c not in front_cols]
df_export = df_dedup[front_cols + remaining_cols].copy()

df_export.to_csv(OUTPUT_CSV, index=False, encoding="utf-8")
OUTPUT_CSV

WindowsPath('c:/Users/JanMc/Dropbox/Education/_GitHub_coursework/janmcconnellCityU-coursework/DS687_CAPSTONE/research/bib_combined_dedup_doi.csv')

## Sampling the de-duplicated dataset

I use sampling to quickly inspect data quality, field completeness, and provenance without reviewing the full dataset.

In [13]:
import pandas as pd
from pathlib import Path

csv_path = Path("bib_combined_dedup_doi.csv")

df_sample_base = pd.read_csv(csv_path)
df_sample_base.shape

(1342, 24)

In [14]:
# Simple random sample
df_sample_base.sample(n=10, random_state=42)

,_doi_norm,_doi_raw,_source_file,_source_file_count,_source_files_all,_quality_score,series,location,keywords,numpages,...,url,address,publisher,isbn,year,title,author,ENTRYTYPE,ID,articleno
802,10.1145/3394486.3403373,10.1145/3394486.3403373,KDD.bib,1,['KDD.bib'],8,KDD '20,"Virtual Event, CA, USA","dynamic graph embedding, heterogeneous graph n...",11,...,https://doi.org/10.1145/3394486.3403373,"New York, NY, USA",Association for Computing Machinery,9781450379984,2020,Dynamic Heterogeneous Graph Neural Network for...,"Luo, Wenjuan and Zhang, Han and Yang, Xiaodi a...",inproceedings,10.1145/3394486.3403373,NaN
548,10.1145/3178876.3186070,10.1145/3178876.3186070,TheWebConf.bib,1,['TheWebConf.bib'],8,WWW '18,"Lyon, France","review usefulness, recommender systems, neural...",10,...,https://doi.org/10.1145/3178876.3186070,"Republic and Canton of Geneva, CHE",International World Wide Web Conferences Steer...,9781450356398,2018,Neural Attentional Rating Regression with Revi...,"Chen, Chong and Zhang, Min and Liu, Yiqun and ...",inproceedings,10.1145/3178876.3186070,NaN
570,10.1145/3219819.3219965,10.1145/3219819.3219965,KDD.bib,1,['KDD.bib'],8,KDD '18,"London, United Kingdom","attention mechanism, deep learning, heterogene...",10,...,https://doi.org/10.1145/3219819.3219965,"New York, NY, USA",Association for Computing Machinery,9781450355520,2018,Leveraging Meta-path based Context for Top- N ...,"Hu, Binbin and Shi, Chuan and Zhao, Wayne Xin ...",inproceedings,10.1145/3219819.3219965,NaN
430,10.1145/2959100.2959170,10.1145/2959100.2959170,RecSys.bib,1,['RecSys.bib'],8,RecSys '16,"Boston, Massachusetts, USA","cold-start, collaborative filtering, explicit ...",8,...,https://doi.org/10.1145/2959100.2959170,"New York, NY, USA",Association for Computing Machinery,9781450340359,2016,Fifty Shades of Ratings: How to Benefit from a...,"Frolov, Evgeny and Oseledets, Ivan",inproceedings,10.1145/2959100.2959170,NaN
259,10.1145/2566486.2568018,10.1145/2566486.2568018,TheWebConf.bib,1,['TheWebConf.bib'],8,WWW '14,"Seoul, Korea","repeat consumption, recency, quality, copying ...",12,...,https://doi.org/10.1145/2566486.2568018,"New York, NY, USA",Association for Computing Machinery,9781450327442,2014,The dynamics of repeat consumption,"Anderson, Ashton and Kumar, Ravi and Tomkins, ...",inproceedings,10.1145/2566486.2568018,NaN
306,10.1145/2661829.2662055,10.1145/2661829.2662055,CIKM.bib,1,['CIKM.bib'],8,CIKM '14,"Shanghai, China","autoregressive models, online serials, popular...",10,...,https://doi.org/10.1145/2661829.2662055,"New York, NY, USA",Association for Computing Machinery,9781450325981,2014,Predicting the Popularity of Online Serials wi...,"Chang, Biao and Zhu, Hengshu and Ge, Yong and ...",inproceedings,10.1145/2661829.2662055,NaN
1108,10.1145/3589335.3648312,10.1145/3589335.3648312,TheWebConf.bib,1,['TheWebConf.bib'],8,WWW '24,"Singapore, Singapore","hierarchical clustering, popularity bias, reco...",10,...,https://doi.org/10.1145/3589335.3648312,"New York, NY, USA",Association for Computing Machinery,9798400701726,2024,Cluster Anchor Regularization to Alleviate Pop...,"Chang, Bo and Meng, Changping and Ma, He and C...",inproceedings,10.1145/3589335.3648312,NaN
503,10.1145/3109859.3109912,10.1145/3109859.3109912,RecSys.bib,1,['RecSys.bib'],8,RecSys '17,"Como, Italy","coverage, learning to rank, long-tail, recomme...",5,...,https://doi.org/10.1145/3109859.3109912,"New York, NY, USA",Association for Computing Machinery,9781450346528,2017,Controlling Popularity Bias in Learning-to-Ran...,"Abdollahpouri, Himan and Burke, Robin and Moba...",inproceedings,10.1145/3109859.3109912,NaN
901,10.1145/3460231.3478515,10.1145/3460231.3478515,RecSys.bib,1,['RecSys.bib'],8,RecSys '21,"Amsterdam, Netherlands","BERT, Embeddings, Fairness, Online Social Netw...",6,...,https://doi.org/10.1145/3460231.3478515,"New York, NY, USA",Association for Computing Machinery,9781450384582,2021,RecSys 2021 Challenge Workshop: Fairness-aware...,"Anelli, Vito Walter and 

In [15]:
cols_to_view = [
    "_doi_norm",
    "title",
    "year",
    "abstract",
    "_source_file_count",
    "_source_files_all",
]

existing_cols = [c for c in cols_to_view if c in df_sample_base.columns]

df_sample_base[existing_cols].sample(n=10, random_state=42)

,_doi_norm,title,year,abstract,_source_file_count,_source_files_all
802,10.1145/3394486.3403373,Dynamic Heterogeneous Graph Neural Network for...,2020,Customer response prediction is critical in ma...,1,['KDD.bib']
548,10.1145/3178876.3186070,Neural Attentional Rating Regression with Revi...,2018,Reviews information is dominant for users to m...,1,['TheWebConf.bib']
570,10.1145/3219819.3219965,Leveraging Meta-path based Context for Top- N ...,2018,Heterogeneous information network (HIN) has be...,1,['KDD.bib']
430,10.1145/2959100.2959170,Fifty Shades of Ratings: How to Benefit from a...,2016,Conventional collaborative filtering technique...,1,['RecSys.bib']
259,10.1145/2566486.2568018,The dynamics of repeat consumption,2014,We study the patterns by which a user consumes...,1,['TheWebConf.bib']
306,10.1145/2661829.2662055,Predicting the Popularity of Online Serials wi...,2014,Recent years have witnessed the rapid prevalen...,1,['CIKM.bib']
1108,10.1145/3589335.3648312,Cluster Anchor Regularization to Alleviate Pop...,2024,Recommender systems are essential for finding ...,1,['TheWebConf.bib']
503,10.1145/3109859.3109912,Controlling Popularity Bias in Learning-to-Ran...,2017,Many recommendation algorithms suffer from pop...,1,['RecSys.bib']
901,10.1145/3460231.3478515,RecSys 2021 Challenge Workshop: Fairness-aware...,2021,The workshop features presentations of accepte...,1,['RecSys.bib']
711,10.1145/3340531.3411993,E-commerce Recommendation with Weighted Expect...,2020,"Different from shopping at retail stores, cons...",1,['CIKM.bib']


In [16]:
df_multi_source = df_sample_base[df_sample_base["_source_file_count"] > 1]

df_multi_source.sample(
    n=min(10, len(df_multi_source)),
    random_state=42
)

,_doi_norm,_doi_raw,_source_file,_source_file_count,_source_files_all,_quality_score,series,location,keywords,numpages,...,url,address,publisher,isbn,year,title,author,ENTRYTYPE,ID,articleno


In [17]:
def is_missing(x):
    if pd.isna(x):
        return True
    return str(x).strip() == ""

missing_title_abstract = df_sample_base[
    (df_sample_base.get("title", "").apply(is_missing)) &
    (df_sample_base.get("abstract", "").apply(is_missing))
]

missing_title_abstract.sample(
    n=min(10, len(missing_title_abstract)),
    random_state=42
)

,_doi_norm,_doi_raw,_source_file,_source_file_count,_source_files_all,_quality_score,series,location,keywords,numpages,...,url,address,publisher,isbn,year,title,author,ENTRYTYPE,ID,articleno
